# Judgement Panda Exfil Activity
Detects Judgement Panda activity as described in Global Threat Report 2019 by Crowdstrike

## Rule Content
```
- title: Judgement Panda Exfil Activity
  id: 03e2746e-2b31-42f1-ab7a-eb39365b2422
  description: Detects Judgement Panda activity as described in Global Threat Report
    2019 by Crowdstrike
  references:
  - https://www.crowdstrike.com/resources/reports/2019-crowdstrike-global-threat-report/
  author: Florian Roth
  date: 2019/02/21
  tags:
  - attack.lateral_movement
  - attack.g0010
  - attack.credential_access
  - attack.t1098
  - attack.exfiltration
  - attack.t1002
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection1:
      CommandLine:
      - '*\ldifde.exe -f -n *'
      - '*\7za.exe a 1.7z *'
      - '* eprod.ldf'
      - '*\aaaa\procdump64.exe*'
      - '*\aaaa\netsess.exe*'
      - '*\aaaa\7za.exe*'
      - '*copy .\1.7z \\*'
      - '*copy \\client\c$\aaaa\\*'
    selection2:
      Image: C:\Users\Public\7za.exe
    condition: selection1 or selection2
  falsepositives:
  - unknown
  level: critical

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_command_line.keyword:(*\\ldifde.exe\ \-f\ \-n\ * OR *\\7za.exe\ a\ 1.7z\ * OR *\ eprod.ldf OR *\\aaaa\\procdump64.exe* OR *\\aaaa\\netsess.exe* OR *\\aaaa\\7za.exe* OR *copy\ .\\1.7z\ \\* OR *copy\ \\client\\c$\\aaaa\\*) OR process_path:"C\:\\Users\\Public\\7za.exe")')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()